In [1]:
import numpy as np
import pandas as pd
import jieba
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [2]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [3]:
news.dropna(inplace=True)

In [4]:
news.replace({'source': r'^(新华社)'},{'source': 1}, regex=True, inplace=True)

In [5]:
news.loc[news['source'] != 1, 'source'] = 0

In [6]:
news = news.reset_index()

In [7]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76925 entries, 0 to 76924
Data columns (total 8 columns):
index      76925 non-null int64
id         76925 non-null int64
author     76925 non-null object
source     76925 non-null int64
content    76925 non-null object
feature    76925 non-null object
title      76925 non-null object
url        76925 non-null object
dtypes: int64(3), object(5)
memory usage: 4.7+ MB


In [8]:
with open('./stop_words.utf8') as f: 
    stop_words =  [l.strip() for l in f]

In [9]:
def cut(string): return [word for word in list(jieba.cut(string)) if word not in stop_words]

In [10]:
all_contents = news['content']

In [11]:
all_contents = [' '.join(cut(s)) for s in all_contents]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_8/f14fxxnn7w13cd0l5x9hl4bm0000gn/T/jieba.cache
Loading model cost 0.859 seconds.
Prefix dict has been built succesfully.


In [12]:
all_contents[0]

'原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 中 裸身 奔走 \r\n 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 \r\n 事发 时 \r\n 到底 发生 \r\n 南 记者 带 一起 还原 现场 \r\n 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 腼腆 90 青年 \r\n 刘青 介绍 月 16 日 早上 时 36 分 正在 环城 南路 附近 值勤 接到 中队 一位 女子 裸身 进入 机动车 可能 危险 警情 骑着 铁骑 沿路 寻找 大概 花 十多分钟 南坪 大道 坂田 出口 龙岗 方向 逆行 辅道 发现 女子 \r\n 女子 身上 一丝不挂 地逆 车流 而行 时 走时 停 时 坐 时 躺 险象环生 刘青停 铁骑 一名 巡防 员 追 上去 发现 女子 情绪 低落 话 刘青 尝试 女子 交流 劝说 女子 离开 女子 愿意 接受 继续 缓慢 地往 南坪 快速路 主干道 走 \r\n 路边 已经 聚集 市民 围观 刺激 女子 情绪 刘青 巡防 员 一边 盯 女子 一边 驱赶 围观 群众 \r\n 现场 还原 \r\n 警方 提供 一份 视频 了解 16 日 早上 时 25 分 女子 出现 坂雪岗 大道 环城 南路 监控 视频 中 女子 穿着 白色 内裤 正 坂雪岗 大道 南坪 快速 方向 缓慢 走 \r\n 当时 上班 高峰期 十字路口 车流 已经 排起 长队 女子 出现 时 路上 市民 纷纷 驻足 观望 不少 车辆 放慢 速度 女子 市民 观望 停下 脚步 依然 缓慢 走 女子 行进 十字路口 中间 时 一辆 货车 挡住 镜头 女子 再次出现 镜头 时 发现 女子 已经 没 穿 内裤 全身 裸露 继续 南坪 快速 方向 走 记者 发现 视频 中 女子 周围 没有 尾随 上前 劝止 市民 \r\n 大清早 路上 看到 情况 \r\n 恐怕 没 办法 淡定 \r\n 面对 这一 情况 \r\n 刘青 表示 根本 不敢 心里 挺 别扭 感觉 尴尬 刘青 跟随 女子 南坪 快速路 主干道 时 女子 作出 意想不到 举动 突然 靠近 护栏 上面 跳下去 刘青 赶忙 冲上去 拉住 女子 手 控制 住 远离

In [13]:
import pickle

In [14]:
with open('news_content_words.pkl', 'wb') as f:
    pickle.dump(all_contents, f)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vectorizer = TfidfVectorizer(max_features=50)

In [17]:
X = vectorizer.fit_transform(all_contents)

In [18]:
X.shape

(76925, 50)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
y = pd.to_numeric(news['source']).values

In [21]:
with open('news_source.pkl', 'wb') as f:
    pickle.dump(y, f)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

In [23]:
y_train.shape

(23077,)

In [24]:
knn_clf = KNeighborsClassifier()

In [25]:
param_grid = [{'n_neighbors': [i for i in range(1,3)]}]

In [26]:
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=3, scoring='accuracy')

In [27]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  2.0min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  2.3min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_neighbors': [1, 2]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=3)

In [28]:
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=3, scoring='accuracy')
print(scores.mean(), scores)

0.9652901550006279 [0.96438321 0.96788872 0.96359854]


In [29]:
scores = cross_val_score(grid_search.best_estimator_, X_test, y_test, cv=3, scoring='accuracy')
print(scores.mean(), scores)

0.9686896283512043 [0.96958217 0.9671848  0.96930191]


In [30]:
knn = grid_search.best_estimator_

In [ ]:
knn.predict(X_test)